In [24]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [25]:
# загружаем данные
df = pd.read_csv('comment_pairs_final.csv')
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['toxic_comment'].tolist(),
    df['neutral_comment'].tolist(),
    test_size=0.1,
    random_state=42
)

In [26]:
# создаем кастомный датасет
class CommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

        assert len(self.texts) == len(self.labels), "Texts and labels must have the same length"

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = str(self.labels[idx])

        # даем четкую инструкцию
        input_text = f"Преобразуй токсичный текст в вежливый: {text}"

        inputs = self.tokenizer.encode_plus(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        labels = self.tokenizer.encode_plus(
            label,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels['input_ids'].squeeze()
        }

In [13]:
# инициализируем модель и токенизатор
model_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/640k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/259M [00:00<?, ?B/s]

In [27]:
# создаем датасеты
train_dataset = CommentDataset(train_texts, train_labels, tokenizer)
val_dataset = CommentDataset(val_texts, val_labels, tokenizer)

In [28]:
# уменьшаем размер batch для экономии памяти
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [29]:
# настройка обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(20100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(20100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [30]:
# устанавливаем learning rate для стабильности
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)  # увеличиваем learning rate

In [34]:
num_epochs = 7

In [35]:
# функция для проверки примеров
def test_model(text):
    model.eval()
    inputs = tokenizer.encode_plus(
        "Преобразуй токсичный текст в вежливый: " + text,
        return_tensors="pt",
        max_length=128,
        padding='max_length',
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=128,
            num_beams=5,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [36]:
# обучение модели
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Эпоха {epoch + 1}/{num_epochs}, Средний loss: {avg_loss:.4f}")

    # Проверяем примеры после каждой эпохи
    if (epoch + 1) % 1 == 0:
        print("\nПроверка примеров:")
        test_text = "Ты полный идиот!"
        result = test_model(test_text)
        print(f"Исходный текст: {test_text}")
        print(f"Нейтральный текст: {result}\n")

Эпоха 1/7, Средний loss: 0.7123

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение о том, что вы имеете в виду.

Эпоха 2/7, Средний loss: 0.6165

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение о человеке вызывает разные мнения.

Эпоха 3/7, Средний loss: 0.5689

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение о человеке отличается от моего.

Эпоха 4/7, Средний loss: 0.5312

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение отличается от моего.

Эпоха 5/7, Средний loss: 0.5016

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение о человеке отличается от моего.

Эпоха 6/7, Средний loss: 0.4732

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение отличается от мнения других.

Эпоха 7/7, Средний loss: 0.4481

Проверка примеров:
Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение о че

In [37]:
# Сохранение модели
model.save_pretrained("detox_model")
tokenizer.save_pretrained("detox_model")

('detox_model/tokenizer_config.json',
 'detox_model/special_tokens_map.json',
 'detox_model/spiece.model',
 'detox_model/added_tokens.json')

In [38]:
# тестируем
test_text = "Ты полный идиот!"
neutral_text = detoxify_text(test_text)
print(f"Исходный текст: {test_text}")
print(f"Нейтральный текст: {neutral_text}")

Исходный текст: Ты полный идиот!
Нейтральный текст: Ваше мнение о человеке отличается от моего.


In [54]:
# тестируем
test_text = "Эта дура даже не знает, как общаться с клиентами. Ни здрасьте, ни до свиданья, обслуживание на нуле!"
neutral_text = detoxify_text(test_text)
print(f"Исходный текст: {test_text}")
print(f"Нейтральный текст: {neutral_text}")

Исходный текст: Эта дура даже не знает, как общаться с клиентами. Ни здрасьте, ни до свиданья, обслуживание на нуле!
Нейтральный текст: Некоторые люди могут не знать, как общаться с клиентами. Это может быть связано с обслуживанием на нуле.


In [39]:
print("Примеры из датасета:")
for i in range(5):
    print(f"Токсичный: {df['toxic_comment'].iloc[i]}")
    print(f"Нейтральный: {df['neutral_comment'].iloc[i]}")
    print()

Примеры из датасета:
Токсичный: Верблюдов-то за что? Дебилы, бл...

Нейтральный: Почему произошла такая ситуация с верблюдами?

Токсичный: Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Нейтральный: Местные особенности восприятия различных культур могут вызывать разные мнения.

Токсичный: Собаке - собачья смерть

Нейтральный: Каждое животное заслуживает достойного обращения и уважения.

Токсичный: Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

Нейтральный: Пожалуйста, обновите страницу. Будет полезно для улучшения работы.

Токсичный: тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)

Нейтральный: Существует множество мнений по вопросу отравления Скрипалей. Вы, возможно, рассматриваете и анализируете различны

In [41]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9f3d5672573bee48cc396da2e5ec72df887f583cf9e10d2ebbf5aa69796b145a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [42]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
def calculate_metrics(original_texts, generated_texts, reference_texts):
    # инициализируем ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []

    for gen, ref in zip(generated_texts, reference_texts):
        # Считаем ROUGE
        rouge_scores = scorer.score(gen, ref)

        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

        # Считаем BLEU
        reference = [ref.split()]
        candidate = gen.split()
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)

    # Считаем средние значения
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"ROUGE-1: {avg_rouge1:.4f}")
    print(f"ROUGE-2: {avg_rouge2:.4f}")
    print(f"ROUGE-L: {avg_rougeL:.4f}")
    print(f"BLEU: {avg_bleu:.4f}")

In [44]:
# загружаем и используем сохраненную модель
model = T5ForConditionalGeneration.from_pretrained("detox_model")
tokenizer = T5Tokenizer.from_pretrained("detox_model")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(20100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(20100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [46]:
# генерируем предсказания для валидационной выборки
generated_texts = []
original_texts = val_texts    # наши токсичные тексты из валидационной выборки
reference_texts = val_labels  # эталонные нейтральные тексты из валидационной выборки

In [47]:
model.eval()
for text in val_texts:
    result = test_model(text)  # используем нашу функцию test_model
    generated_texts.append(result)

In [48]:
def calculate_metrics(original_texts, generated_texts, reference_texts):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []

    for gen, ref in zip(generated_texts, reference_texts):
        # считаем ROUGE
        rouge_scores = scorer.score(gen, ref)

        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

        # считаем BLEU
        reference = [ref.split()]
        candidate = gen.split()
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)

    # считаем средние значения
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"ROUGE-1: {avg_rouge1:.4f}")
    print(f"ROUGE-2: {avg_rouge2:.4f}")
    print(f"ROUGE-L: {avg_rougeL:.4f}")
    print(f"BLEU: {avg_bleu:.4f}")

In [49]:
# вычисляем метрики
calculate_metrics(original_texts, generated_texts, reference_texts)

ROUGE-1: 0.0209
ROUGE-2: 0.0037
ROUGE-L: 0.0209
BLEU: 0.0077


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [53]:
import random

random_indices = random.sample(range(len(original_texts)), 5)

for i in random_indices:
    print(f"\nИсходный текст: {original_texts[i]}")
    print(f"Сгенерированный текст: {generated_texts[i]}")
    print(f"Эталонный текст: {reference_texts[i]}")


Исходный текст: Во всем прав. До сих пор он приносит ТОННЫ АНАЛЬНОЙ БОЛИ всем сортам срынкоскота, от нацидауна до либералошвали. Особенно сильно с него горят различные лживые творческие илитки, изрыгая тонны пиздежа про миллиарда расстрелянных, т.к. они не могли при нем поплевывать в холопов. Полыхания этих животных наблюдать приятнее всего.

Сгенерированный текст: Существуют разные мнения о том, что его качество может быть воспринято как агрессивное. Важно учитывать различные точки зрения и подходы к этому вопросу.
Эталонный текст: Этот комментарий выражает мнение о том, что некоторые группы людей испытывают негативные эмоции по отношению к определённой личности. Некоторые представители творческой элиты критикуют его за различные заявления.

Исходный текст: Да и ладно, зато хоть на женщину похожа, а не на швабру с костями...

Сгенерированный текст: Некоторые люди могут иметь разные мнения о женщинах, а не на женщине.
Эталонный текст: Комментарий о внешнем виде человека может быть вос